# Step 1 <br>
> download the "20 News Group" dataset from the http://qwone.com/~jason/20Newsgroups/ website.
>Note: please download the following dataset to maintain the consistency.
> 20news-bydate.tar.gz-20NewsGroups sorted by date; duplicats and som headers removed(18846 documents)







In [2]:
# some import imports

import pandas as pd                                                    # for create df and write to csv files
import shutil                                                          #copy files from one directory to anothers
import glob                                                            # access to directories
import os, os.path,re 
import random                                                          # for suffering the list
import nltk 
from sklearn.feature_extraction.text import CountVectorizer            #for create feature matrix
from nltk.corpus import stopwords                                      # english stop words
from nltk import word_tokenize
import string
from nltk.corpus import words
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/rasika/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/rasika/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/rasika/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/rasika/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/rasika/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Step 2<br>
> Create five main classes by combining the document in the subclasses in the both training and testing folders.
> For examples combine the document in the comp.graphics, comp.os.ms-windows.mics,comp.sys.ibm.pc.hardware, comp.sys.mac.hardware and comp.windows.X classes create one document set for the<b> computer</b> class.Similary, create <b>recreational, science and talk classes </b>





In [3]:
classes = ['computer','recreational','science', 'talk']

In [4]:
root = './classes/'

In [5]:
for class_file in classes:
    p = os.path.join(root,class_file)
    for file_name in glob.iglob('20news-bydate*/'+class_file[:3]+'*/*', recursive=True):
        shutil.copy(file_name, p)
    
    print(class_file +" files: " ,len([name for name in os.listdir(p) if os.path.isfile(os.path.join(p, name))]))

computer files:  4891
recreational files:  3116
science files:  3951
talk files:  3253


# Step 3<br>
>split douments randomly in each class into 2 separate dataset, i.e train set and test set.
Note: divied the document set in each class with 70(train) & 30(test) ratio

In [6]:
def split_datafiles(files):
    random.Random(101).shuffle(files)
    train_split = int(0.7 * len(files))
    test_split = int(0.3 * len(files))
    train_files=files[:train_split]
    test_files=files[-test_split:]
    return train_files,test_files # return the list of files names train and test

# step 4<br>
>Pre-process each document and transform it to a feature vector. Finally, save all the document into multidimensional array in python. Then ,save each class into 2 separate CSV files. For examples, computer_train.csv and computer_test.csv


In [7]:
stop=["a", "about", "above", "above", "across", "after", 
       "afterwards", "again", "against", "all", "almost", "alone", 
       "along", "already", "also","although","always","am","among", 
       "amongst", "amoungst", "amount",  "an", "and", "another", "any",
       "anyhow","anyone","anything","anyway", "anywhere", "are", "around",
       "as",  "at", "back","be","became", "because","become","becomes", 
       "becoming", "been", "before", "beforehand", "behind", "being", 
       "below", "beside", "besides", "between", "beyond", "bill", "both",
       "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", 
       "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", 
       "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", 
       "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
       "everything", "everywhere", "except", "few", "fifteen", "fify", "fill",
       "find", "fire", "first", "five", "for", "former", "formerly", "forty", 
       "found", "four", "from", "front", "full", "further", "get", "give", "go", 
       "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", 
       "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
       "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest",
       "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", 
       "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", 
       "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", 
       "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", 
       "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", 
       "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", 
       "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
       "out", "over", "own","part", "per", "perhaps", "please", "put",
       "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", 
       "serious", "several", "she", "should", "show", "side", "since", 
       "sincere", "six", "sixty", "so", "some", "somehow", "someone", 
       "something", "sometime", "sometimes", "somewhere", "still", "such", 
       "system", "take", "ten", "than", "that", "the", "their", "them", 
       "themselves", "then", "thence", "there", "thereafter", "thereby",
       "therefore", "therein", "thereupon", "these", "they", "thickv",
       "thin", "third", "this", "those", "though", "three", "through", 
       "throughout", "thru", "thus", "to", "together", "too", "top", 
       "toward", "towards", "twelve", "twenty", "two", "un", "under",
       "until", "up", "upon", "us", "very", "via", "was", "we", "well",
       "were", "what", "whatever", "when", "whence", "whenever", "where",
       "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", 
       "whether", "which", "while", "whither", "who", "whoever", "whole", "whom",
       "whose", "why", "will", "with", "within", "without", "would", "yet", "you",
       "your", "yours", "yourself", "yourselves", "the"] 

In [8]:
stop_words = set(stopwords.words('english') + list(string.punctuation) + stop) # english stop words

In [9]:
word_list = words.words()
stemmer = nltk.stem.WordNetLemmatizer()

In [10]:
def getFileContent(path,file_name):
    with open(path+"/"+file_name, 'r', encoding = "ISO-8859-1") as theFile:
        data = theFile.read().replace('\n', '')
        return data

In [12]:
def getFileWordsList(p,files_name):
    words=[]
    
    for file_name in files_name:
        text = getFileContent(p,file_name)
        words+= [stemmer.lemmatize(i) for i in word_tokenize(text.lower()) if i not in stop_words and i in word_list
                and len(i) > 1] 
    
    return list(set(words))

In [15]:
def getUniqueWordList():
    uWords_train = []
    uWords_test = []
    
    for class_file in classes:
        p = os.path.join(root,class_file)
        files= [name for name in os.listdir(p) if os.path.isfile(os.path.join(p, name))]
        train_files,test_files = split_datafiles(files)
        print("progress "+ class_file)
        uWords_train += getFileWordsList(p,train_files)
        print(' number of unique words '+ class_file +' train is :', len(set(uWords_train)))
        uWords_test += getFileWordsList(p,test_files)
        print(' number of unique words '+ class_file +' test is : ', len(set(uWords_test)))
      
    return list(set(uWords_train)), list(set(uWords_test))

In [16]:
uniq_words_train, uniq_words_test = getUniqueWordList()

progress computer
 number of unique words computer train is : 8569
 number of unique words computer test is :  9521
progress recreational
 number of unique words recreational train is : 12608
 number of unique words recreational test is :  13288
progress science
 number of unique words science train is : 17318
 number of unique words science test is :  18084
progress talk
 number of unique words talk train is : 20894
 number of unique words talk test is :  21476


In [17]:
cv_train = CountVectorizer(vocabulary=uniq_words_train)
cv_test = CountVectorizer(vocabulary=uniq_words_test)

In [18]:
def create_feature_vector(root,class_file):
    
    p = os.path.join(root,class_file)
    files= [name for name in os.listdir(p) if os.path.isfile(os.path.join(p, name))]
    train_files,test_files = split_datafiles(files)
    text_files=[]
    for file in train_files:
        text_files.append(getFileContent(p,file))
        
    X = cv_train.fit_transform(text_files)
    Y = cv_train.get_feature_names()
    df = pd.DataFrame(data=X.toarray(), columns=Y)
    df.to_csv('./csv/'+class_file+'_train.csv')
    print('created '+class_file+' train csv')
    
    text_files=[]
    for file in test_files:
        text_files.append(getFileContent(p,file))
            
    X = cv_test.fit_transform(text_files)
    Y = cv_test.get_feature_names()
    df = pd.DataFrame(data=X.toarray(), columns=Y)
    df.to_csv('./csv/'+class_file+'_test.csv')
    print('created '+class_file+' test csv')

In [19]:
create_feature_vector(root,classes[0])
create_feature_vector(root,classes[1])
create_feature_vector(root,classes[2])
create_feature_vector(root,classes[3])

created computer train csv
created computer test csv
created recreational train csv
created recreational test csv
created science train csv
created science test csv
created talk train csv
created talk test csv
